In [111]:
from csv import writer
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

 # list of column names
field_names = ['formatted_date', 'high', 'low',
            'open', 'close', 'volume', 'adjclose']
            
pre_day = 1

MA_1 = 7
MA_2 = 14
MA_3 = 21

cols_y_close = ['close']
cols_y_open = ['open']
cols_y_high = ['high']
cols_y_low = ['low']



cols_x_close = ['low', 'open', 'high', 'H-L', f'SMA_{MA_1}_close', f'SMA_{MA_2}_close', f'SMA_{MA_3}_close', f'SD_{MA_1}_close',f'SD_{MA_3}_close']
cols_x_open = ['high', 'low', 'close', 'H-L', f'SMA_{MA_1}_open', f'SMA_{MA_2}_open', f'SMA_{MA_3}_open', f'SD_{MA_1}_open',f'SD_{MA_3}_open']
cols_x_high = ['low', 'open', 'close', 'O-C', f'SMA_{MA_1}_high', f'SMA_{MA_2}_high', f'SMA_{MA_3}_high', f'SD_{MA_1}_high',f'SD_{MA_3}_high']
cols_x_low = ['high', 'open', 'close', 'O-C', f'SMA_{MA_1}_low', f'SMA_{MA_2}_low', f'SMA_{MA_3}_low', f'SD_{MA_1}_low',f'SD_{MA_3}_low']

scala_x = MinMaxScaler(feature_range=(0,1))
scala_y = MinMaxScaler(feature_range=(0,1))


In [112]:
def loadModel():
    model_close = load_model("./models/GRU/GRU_CLOSE.h5")
    model_open = load_model("./models/GRU/GRU_OPEN.h5")
    model_high = load_model("./models/GRU/GRU_HIGH.h5")
    model_low = load_model("./models/GRU/GRU_LOW.h5")
    return model_close,model_open ,model_high ,model_low


def loadModelError():
    model_error_close = pickle.load(open("./models/ELASTICNET/elasticnet_close.h5",'rb'))
    model_error_open = pickle.load(open("./models/ELASTICNET/elasticnet_open.h5",'rb'))
    model_error_high = pickle.load(open("./models/ELASTICNET/elasticnet_high.h5",'rb'))
    model_error_low = pickle.load(open("./models/ELASTICNET/elasticnet_low.h5",'rb'))
    return model_error_close,model_error_open ,model_error_high ,model_error_low
def processingData(df,cols_y_close,cols_y_open,cols_y_high,cols_y_low):

    scaled_data_x_close = scala_x.fit_transform(df[cols_x_close].values.reshape(-1, len(cols_x_close)))
    scaled_data_x_open = scala_x.fit_transform(df[cols_x_open].values.reshape(-1, len(cols_x_open)))
    scaled_data_x_high = scala_x.fit_transform(df[cols_x_high].values.reshape(-1, len(cols_x_high)))
    scaled_data_x_low = scala_x.fit_transform(df[cols_x_low].values.reshape(-1, len(cols_x_low)))

    scaled_data_y_close = scala_y.fit_transform(df[cols_y_close].values.reshape(-1, len(cols_y_close)))
    scaled_data_y_open = scala_y.fit_transform(df[cols_y_open].values.reshape(-1, len(cols_y_open)))
    scaled_data_y_high = scala_y.fit_transform(df[cols_y_high].values.reshape(-1, len(cols_y_high)))
    scaled_data_y_low = scala_y.fit_transform(df[cols_y_low].values.reshape(-1, len(cols_y_low)))
    return scaled_data_x_close,scaled_data_x_open,scaled_data_x_high,scaled_data_x_low,scaled_data_y_close,scaled_data_y_open,scaled_data_y_high,scaled_data_y_low,scala_x,scala_y

def predict(df, cols_x, model,error_model):
    pre_day = 1
    scaled_data_x = scala_x.fit_transform(df[len(df)-pre_day:][cols_x].values.reshape(-1, len(cols_x)))  
    x_last=np.array([scaled_data_x])
  
    predict = model.predict(x_last)
    predict_price = scala_y.inverse_transform(predict)
    print(predict_price)
    # predict_error= error_model.predict(np.array(predict_price).reshape(-1,1))[0]
    return predict_price


In [113]:
model_close,model_open ,model_high ,model_low=loadModel()
model_error_close,model_error_open ,model_error_high ,model_error_low= loadModelError()


df3 = pd.read_csv("./btc_pred_process.csv")
scaled_data_x_close,scaled_data_x_open,scaled_data_x_high,scaled_data_x_low,scaled_data_y_close,scaled_data_y_open,scaled_data_y_high,scaled_data_y_low,scala_x,scala_y=processingData(df3,cols_y_close,cols_y_open,cols_y_high,cols_y_low)




In [114]:
# low_pred = predict(df3, cols_x_low, model_low,model_error_low) 
close_pred = predict(df3, cols_x_close, model_close,model_error_close) 
# open_pred = predict(df3, cols_x_open, model_open,model_error_open) 
# high_pred = predict(df3, cols_x_high, model_high,model_error_high) 

# print(low_pred,close_pred,open_pred,high_pred)

1/1 [==============================] - 1s 1s/step
[[3289.174]]
